#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from IPython.display import HTML

In [2]:
# to show full text in dataframe
pd.set_option('display.max_colwidth', None)

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data looks like in the **event_datafile_new.csv**:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance the local machine 
# (127.0.0.1:9042)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, we need a session
session = cluster.connect()

In [7]:
# define a row factory to get results in nice tabular form
def pandas_row_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_row_factory

#### Create Keyspace

In [8]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = {'class' : 'SimpleStrategy',
                        'replication_factor' : 1}"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [10]:
# helper functions
def drop_table(table_name, session=session):
    """Drop the provided table if it exists.
        Session refers to the active Cassandra session.
    """
    dropTable = f"DROP TABLE IF EXISTS {table_name}"
    try:
        session.execute(dropTable)
    except Exception as e:
        print(e)


def create_table(query_string, session=session):
    """Create table based on the provided query string.
        Session refers to the active Cassandra session.
    """
    try:
        session.execute(query_string)
    except Exception as e:
        print(e)
        

def run_query(query_string, col_names, query_desc, session=session):
    """Given the query string, execute it.
        For pretty print, provide custom column names and a text briefly  explaining
        what the query does.
        Session refers to the active Cassandra session.
    """
    try:
        rows = session.execute(query_string)
    except Exception as e:
        print(e)
    else:
        # prepare results
        df_ = rows._current_rows
        df_.columns = col_names

        print(query_desc)
        return HTML(df_.to_html(index=False))

### We create queries to answer the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

###### Query 1

In [11]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = """
    SELECT 
        artist
        , song
        , length
    FROM artist_song_by_session
    WHERE sessionId=338 
        AND itemInSession=4
"""


# based on the query, define a table artist_song_by_session
# select the artist, song and length columns as per requirement
# since we are intereseted in results based on the session-id and itemsInSession, these two are selected as 
# the primary key
# the query result is then sorted alphabetically on the artist name (clustering key)
createTable = """
    CREATE TABLE IF NOT EXISTS artist_song_by_session (
        sessionId INT
        , itemInSession INT
        , artist TEXT
        , song TEXT
        , length DECIMAL
        , PRIMARY KEY (
            (sessionId, itemInSession)
            , artist
        )
    )
"""

In [12]:
# drop table at first if required, e.g. due to change in table structure
drop_table('artist_song_by_session')

In [13]:
# create table
create_table(createTable)

In [14]:
# read and insert data into table
file = 'event_datafile_new.csv'

query = '' # 'reset' query string variable
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),    # convert sessionId to int
                                int(line[3]),    # convert itemInSession to int
                                line[0],
                                line[9],
                                float(line[5]))) # convert length to float

In [15]:
col_names = ['Artist', 'Song Title', 'Song Length']
query_desc = 'Name of artist and song title for session ID: 338 and Item in session: 4'
run_query(query1, col_names, query_desc)

Name of artist and song title for session ID: 338 and Item in session: 4


Artist,Song Title,Song Length
Faithless,Music Matters (Mark Knight Dub),495.3073


##### Query 2

In [16]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = """
    SELECT
        artist
        , song
        , firstName
        , lastName
    FROM user_session
    WHERE userId=10
        AND sessionId=182
"""       

# based on the query, define a table user_session
# select the artist, song and user first and last name as per requirement
# since we are intereseted in results based on the user and session-id, these two are selected as the primary key
# the query result is then sorted by itemInSession as per requirement
createTable = """
    CREATE TABLE IF NOT EXISTS user_session (
        userId INT
        , sessionId INT
        , itemInSession INT
        , artist TEXT
        , song TEXT
        , firstName TEXT
        , lastName TEXT
        , PRIMARY KEY (
            (userId, sessionId)
            , itemInSession
        )
    )
"""             

In [17]:
# drop table at first if required, e.g. due to change in table structure
drop_table('user_session')

In [18]:
# create table
create_table(createTable)

In [19]:
# read and insert data into table
file = 'event_datafile_new.csv'

query = '' # 'reset' query string variable
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),   # convert userId to int
                                int(line[8]),    # convert sessionId to int
                                int(line[3]),    # convert itemInSession to int
                                line[0],
                                line[9],
                                line[1],
                                line[4])) 

In [20]:
col_names = ['Artist', 'Song Title', 'First Name', 'Last Name']
query_desc = 'All songs played by the user with Id 10 and session-id 182'
run_query(query2, col_names, query_desc)

All songs played by the user with Id 10 and session-id 182


Artist,Song Title,First Name,Last Name
Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
Three Drives,Greece 2000,Sylvie,Cruz
Sebastien Tellier,Kilometer,Sylvie,Cruz
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Sylvie,Cruz


##### Query 3

In [21]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the 
## song 'All Hands Against His Own'
query3 = """
    SELECT
        firstName
        , lastName
    FROM user_song
    WHERE song='All Hands Against His Own'
"""   

# based on the query, define a table user_song
# select only user first and last name as per requirement
# since we are intereseted in results based on the song title, only the song title is the primary key
# use the userId as the clustering key

# the query result is then sorted by first and last name
createTable = """
    CREATE TABLE IF NOT EXISTS user_song (
         song TEXT
        , userId INT 
        , firstName TEXT
        , lastName TEXT
        , PRIMARY KEY (
            (song)
            , userId
        )
    )
"""       

In [22]:
# drop table at first if required, e.g. due to change in table structure
drop_table('user_song')

In [23]:
# create table
create_table(createTable)

In [24]:
# read and insert data into table
file = 'event_datafile_new.csv'

query = '' # 'reset' query string variable
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], 
                                int(line[10]),
                                line[1],
                                line[4])) 

In [25]:
col_names = ['First Name', 'Last Name']
query_desc = 'All users who listened to the song "All Hands Against His Own"'
run_query(query3, col_names, query_desc)

All users who listened to the song "All Hands Against His Own"


First Name,Last Name
Jacqueline,Lynch
Tegan,Levine
Sara,Johnson


### Drop the tables before closing out the sessions

In [26]:
## Drop the table before closing out the sessions
drop_table('artist_song_by_session')

drop_table('user_session')

drop_table('user_song')

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()